### 1. Find the most popular category in each country
Find the most popular Pinterest category people post to based on their country.

Your query should return a DataFrame that contains the following columns:

- country
- category
- category_count, a new column containing the desired query output

In [ ]:
sql_df1 = spark.sql("""
                WITH category_totals AS (
                    SELECT
                        country,
                        category,
                        COUNT(category) as category_count
                    FROM
                        hilla_pin
                    INNER JOIN
                        hilla_geo ON hilla_pin.ind = hilla_geo.ind
                    GROUP BY
                        country,
                        category
                ), category_ranks AS (
                    SELECT
                        country,
                        category,
                        category_count,
                        RANK() OVER (
                            PARTITION BY country
                            ORDER BY category_count DESC
                        ) as category_rank
                    FROM category_totals
                )
                SELECT
                    country,
                    category,
                    category_count
                FROM
                    category_ranks
                WHERE
                    category_rank = 1
                ORDER BY country;
                        """)

sql_df1.write.mode("overwrite").parquet("/tmp/parquet/most_popular_category_per_country")

### 2. Find which was the most popular category each year

Find how many posts each category had between 2018 and 2022.

Your query should return a DataFrame that contains the following columns:
- post_year, a new column that contains only the year from the timestamp column
- category
- category_count, a new column containing the desired query output

In [ ]:
sql_df2 = spark.sql("""
            FROM (
                SELECT
                    date_part('year', timestamp) as post_year,
                    category,
                    COUNT(category) as category_count
                FROM
                    hilla_pin
                INNER JOIN
                    hilla_geo ON hilla_pin.ind = hilla_geo.ind
                GROUP BY
                    post_year,
                    category
            )
            SELECT
                post_year,
                category,
                category_count
            WHERE
                post_year BETWEEN 2018 and 2022
            ORDER BY
                post_year,
                category_count DESC;
                        """)
sql_df2.write.mode("overwrite").parquet("/tmp/parquet/most_popular_category_per_year")

### 3. Find the user with most followers in each country
Step 1: For each country find the user with the most followers.

Your query should return a DataFrame that contains the following columns:

- country
- poster_name
- follower_count

Step 2: Based on the above query, find the country with the user with most followers.

Your query should return a DataFrame that contains the following columns:

- country
- follower_count
This DataFrame should have only one entry

In [ ]:
sql_df3_1 = spark.sql("""
                WITH all_posters AS (
                    SELECT
                        country,
                        poster_name,
                        follower_count
                    FROM
                        hilla_pin
                    LEFT JOIN
                        hilla_geo ON hilla_pin.ind = hilla_geo.ind
                ), follower_counts_ranked AS (
                    SELECT
                        country,
                        poster_name,
                        follower_count,
                        RANK() OVER (
                            PARTITION BY COUNTRY
                            ORDER BY follower_count DESC
                        ) follower_count_ranking
                    FROM all_posters
                )
                SELECT DISTINCT
                    country,
                    poster_name,
                    follower_count
                FROM
                    follower_counts_ranked
                WHERE
                    follower_count_ranking = 1
                ORDER BY
                    country;
                        """)

sql_df3_1.write.mode("overwrite").parquet("/tmp/parquet/user_with_most_followers_per_country")

In [ ]:
sql_df3_1.createOrReplaceTempView("hilla_sql_df3_1")

sql_df3_2 = spark.sql("""
                SELECT
                    country,
                    follower_count
                FROM (
                    SELECT
                        country,
                        follower_count,
                        RANK() OVER (
                            ORDER BY follower_count DESC
                        ) follower_count_ranking
                    FROM
                        hilla_sql_df3_1
                    )
                WHERE follower_count_ranking = 1;
                    """)
sql_df3_2.write.mode("overwrite").parquet("/tmp/parquet/country_of_user_with_most_followers")

### 4. Find the most popular category for different age groups.

What is the most popular category people post to based on the following age groups:

- 18-24
- 25-35
- 36-50
- +50

Your query should return a DataFrame that contains the following columns:

- age_group, a new column based on the original age column
- category
- category_count, a new column containing the desired query output

In [ ]:
sql_df4 = spark.sql("""
            WITH category_counts AS (
                SELECT
                    category,
                    COUNT(category) as category_count,
                    CASE
                        WHEN age > 50 THEN '50+'
                        WHEN age > 35 THEN '36-50'
                        WHEN age > 25 THEN '25-35'
                        WHEN age > 17 THEN '18-24'
                        ELSE 'invalid age' 
                    END AS age_group
                FROM
                    hilla_pin
                LEFT JOIN
                    hilla_user ON hilla_pin.ind = hilla_user.ind
                GROUP BY
                    age_group,
                    category
            ), category_count_rankings AS (
                SELECT
                    age_group,
                    category,
                    category_count,
                    RANK() OVER (
                        PARTITION BY age_group
                        ORDER BY category_count DESC
                    ) category_count_ranking
                FROM
                    category_counts
            )
            SELECT
                age_group,
                category,
                category_count
            FROM
                category_count_rankings
            WHERE
                category_count_ranking = 1;
                    """)

sql_df4.write.mode("overwrite").parquet("/tmp/parquet/most_popular_category_per_age_group")

### 5. Find the median follower count for different age groups

What is the median follower count for users in the following age groups:

18-24
25-35
36-50
+50

Your query should return a DataFrame that contains the following columns:

- age_group, a new column based on the original age column
- median_follower_count, a new column containing the desired query output

In [ ]:
sql_df5 = spark.sql("""
            SELECT
                CASE
                    WHEN age > 50 THEN '50+'
                    WHEN age > 35 THEN '36-50'
                    WHEN age > 25 THEN '25-35'
                    WHEN age > 17 THEN '18-24'
                    ELSE 'invalid age' 
                END AS age_group,
                percentile_approx(follower_count, 0.5) AS median_follower_count
            FROM
                hilla_pin
            LEFT JOIN
                hilla_user ON hilla_pin.ind = hilla_user.ind
            GROUP BY
                age_group
            ORDER BY
                age_group;
            """)

sql_df5.write.mode("overwrite").parquet("/tmp/parquet/median_follower_count_per_age_group")

### 6. Find how many users have joined each year

Find how many users have joined between 2015 and 2020.

Your query should return a DataFrame that contains the following columns:

- post_year, a new column that contains only the year from the timestamp column
- number_users_joined, a new column containing the desired query output

In [ ]:
sql_df6 = spark.sql("""
            FROM (
                SELECT
                    date_part('year', date_joined) as post_year,
                    COUNT(ind) as number_users_joined
                FROM
                    hilla_user
                GROUP BY
                    post_year
                )
            SELECT
                post_year,
                number_users_joined
            WHERE
                post_year BETWEEN 2015 and 2022
            ORDER BY
                post_year;
                        """)

sql_df6.write.mode("overwrite").parquet("/tmp/parquet/new_users_per_year")

### 7. Find the median follower count of users based on their joining year

Find the median follower count of users have joined between 2015 and 2020.

Your query should return a DataFrame that contains the following columns:

- post_year, a new column that contains only the year from the timestamp column
- median_follower_count, a new column containing the desired query output

In [ ]:
sql_df7 = spark.sql("""
        FROM (
            SELECT
                date_part('year', date_joined) as post_year,
                follower_count
            FROM
                hilla_user
            LEFT JOIN
                hilla_pin ON hilla_user.ind = hilla_pin.ind
            )
        SELECT
            post_year,
            percentile_approx(follower_count, 0.5) AS median_follower_count
        WHERE
            post_year BETWEEN 2015 AND 2020
        GROUP BY
            post_year
        ORDER BY
            post_year;
                """)

sql_df7.write.mode("overwrite").parquet("/tmp/parquet/median_follower_count_per_joining_year")

### 8. Find the median follower count of users based on their joining year and age group

Find the median follower count of users that have joined between 2015 and 2020, based on which age group they are part of.

Your query should return a DataFrame that contains the following columns:

- age_group, a new column based on the original age column
- post_year, a new column that contains only the year from the timestamp column
- median_follower_count, a new column containing the desired query output

In [ ]:
sql_df8 = spark.sql("""
            SELECT
                CASE
                    WHEN age > 50 THEN '50+'
                    WHEN age > 35 THEN '36-50'
                    WHEN age > 25 THEN '25-35'
                    WHEN age > 17 THEN '18-24'
                    ELSE 'invalid age' 
                END AS age_group,
                date_part('year', date_joined) as post_year,
                percentile_approx(follower_count, 0.5) AS median_follower_count
            FROM
                hilla_pin
            RIGHT JOIN
                hilla_user ON hilla_pin.ind = hilla_user.ind
            GROUP BY
                age_group,
                post_year
            ORDER BY
                post_year,
                age_group;
                    """)

sql_df8.write.mode("overwrite").parquet("/tmp/parquet/median_follower_count_per_joining_year_and_age_group")